# Defining a POMDP with the Generative Interface

In this tutorial we will define a version of the Crying Baby Problem [1] with the generative interface of POMDPs.jl. To find out more about the generative interface, please see this section of the POMDPs.jl documentation: [Generative POMDP Interface](http://juliapomdp.github.io/POMDPs.jl/latest/generative.html#generative_doc-1).

[1] Kochenderfer, Mykel J. Decision Making Under Uncertainty: Theory and Application. MIT Press, 2015

In [4]:
using POMDPs
using Random # for AbstractRNG

## Model type

We begin by implementing the type that represents the model itself. First, some notes about the Crying Baby POMDP:

- The *state* is a Boolean value representing whether the baby is hungry
- The *action* is a Boolean indicateting whether the agent is feeding the baby
- The *observation* is a Boolean indicating whether the baby is crying
- When the baby is hungry, it cries with a high probability and cries with a low probability when it is full
- When the baby is fed, it immediately becomes full.
- At each step, the baby has a small probability of becoming hungry.

The `BabyPOMDP` model type is a subtype of the [`POMDP`](http://juliapomdp.github.io/POMDPs.jl/latest/api.html#POMDPs.POMDP) abstract type. `POMDP` has three parameters which indicate the types used to represent the state, action, and observation respectively, which are all `Bool` in this case. The type also includes parameters that will be used in the reward and transition definitions.

In [5]:
struct BabyPOMDP <: POMDP{Bool, Bool, Bool}
    r_feed::Float64
    r_hungry::Float64
    p_become_hungry::Float64
    p_cry_when_hungry::Float64
    p_cry_when_not_hungry::Float64
    discount::Float64   
end

BabyPOMDP() = BabyPOMDP(-5., -10., 0.1, 0.8, 0.1, 0.9);

## State transitions

Next we describe the *state transitions*. The [`generate_s`](http://juliapomdp.github.io/POMDPs.jl/latest/api.html#POMDPs.generate_s) function controls how the state transitions by generating a state for the next time step given the current state (`s`) and action (`a`).

The `rng` argument is a random number generator that should be used to generate all random numbers within the function for the sake of reproducibility. To find out more about random number generation in POMDPs.jl, consult the documentation [TODO: add link].

In [6]:
function POMDPs.generate_s(p::BabyPOMDP, s::Bool, a::Bool, rng::AbstractRNG)
    if s # hungry
        return true
    else # not hungry
        return rand(rng) < p.p_become_hungry ? true : false
    end
end